In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('strike_reports.csv', encoding='latin1')

In [3]:
df.sample(2)

,INDEX_NR,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME,TIME_OF_DAY,AIRPORT_ID,AIRPORT,LATITUDE,LONGITUDE,...,ENROUTE_STATE,NR_INJURIES,NR_FATALITIES,COMMENTS,REPORTED_NAME,REPORTED_TITLE,SOURCE,PERSON,LUPDATE,TRANSFER
243352,1199503,3/20/2021 0:00:00,3,2021,13:17,Day,KMCO,ORLANDO INTL,28.42889,-81.31603,...,NaN,NaN,NaN,"ADDED AOS, HEIGHT, SPEED AND FUSELAGE PER MAIL...",REDACTED,REDACTED,Multiple,Airport Operations,2/25/2022 0:00:00,0
213052,1033346,8/4/2020 0:00:00,8,2020,07:30,NaN,KDEN,DENVER INTL AIRPORT,39.85841,-104.66700,...,NaN,NaN,NaN,NaN,REDACTED,REDACTED,FAA Form 5200-7-E,Carcass Found,9/2/2020 0:00:00,0


In [4]:
# make column names lowercase
df.columns = df.columns.str.lower()

In [5]:
# make the date column a datetime and set it as index
df = df.rename(columns={'incident_date' : 'date'})

# tried to make datetime conversion faster, but I ran into problems and it really wasn't that slow
# df.date = df.date.str.rstrip('0:00:00')
df.date = df.date.astype('datetime64')
df = df.set_index('date')
df = df.sort_index()

In [6]:
# confirm the month and year columns are the same as the index datetime
df[df.index.month != df.incident_month]

,index_nr,incident_month,incident_year,time,time_of_day,airport_id,airport,latitude,longitude,runway,...,enroute_state,nr_injuries,nr_fatalities,comments,reported_name,reported_title,source,person,lupdate,transfer
date,,,,,,,,,,,,,,,,,,,,,


In [7]:
df[df.index.year != df.incident_year]

,index_nr,incident_month,incident_year,time,time_of_day,airport_id,airport,latitude,longitude,runway,...,enroute_state,nr_injuries,nr_fatalities,comments,reported_name,reported_title,source,person,lupdate,transfer
date,,,,,,,,,,,,,,,,,,,,,


In [8]:
# confirm all the nulls for state are the same as the nulls for faaregion
df[(df.state.isnull()) & (df.faaregion.isnull())].shape

(34384, 99)

In [9]:
# state is null if airport_id is 'ZZZZ', 
# AND there are 82 additional rows where state is null and airport_id is NOT 'ZZZZ'
df[(df.state.isnull()) & (df.airport_id != 'ZZZZ')]

,index_nr,incident_month,incident_year,time,time_of_day,airport_id,airport,latitude,longitude,runway,...,enroute_state,nr_injuries,nr_fatalities,comments,reported_name,reported_title,source,person,lupdate,transfer
date,,,,,,,,,,,,,,,,,,,,,
1990-03-31,637715,3,1990,17:30,NaN,PVT,PRIVATE AIR STRIP,NaN,NaN,NaN,...,NaN,NaN,NaN,NTSB LAX90LA151 STRIKE WAS DELETED BY ERROR BT...,REDACTED,REDACTED,NTSB,Other,5/18/2015 0:00:00,0
1990-08-10,1083895,8,1990,NaN,Day,H2O,REMOTE_WATER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,REDACTED,REDACTED,FAA Form 5200-7,NaN,1/11/2021 0:00:00,0
1991-12-21,612244,12,1991,NaN,Day,RIGG,OIL RIG,NaN,NaN,NaN,...,NaN,NaN,NaN,/Legacy Record=104010/,REDACTED,REDACTED,FAA Form 5200-7,Pilot,4/1/1997 0:00:00,0
1992-05-30,637035,5,1992,NaN,NaN,PVT,PRIVATE AIR STRIP,NaN,NaN,NaN,...,NaN,NaN,NaN,(UPDATED LOCATION) /Legacy Record=128886/,REDACTED,REDACTED,NTSB,Other,5/18/2015 0:00:00,0
1993-09-20,636076,9,1993,09:45,Day,PVT,PRIVATE AIR STRIP,NaN,NaN,NaN,...,NaN,NaN,NaN,"SOURCE = 5200-7, NTSB SEA93LA203 & AAIPN OPER...",REDACTED,REDACTED,Multiple,Pilot,5/18/2015 0:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-10,1406874,1,2023,07:37,Dawn,NaN,JOHN GLENN COLUMBUS INTL ARPT - OH,NaN,NaN,28L,...,NaN,NaN,NaN,*** Strike Report: 2023-01-10-095944-R1( Repor...,REDACTED,REDACTED,FAA Form 5200-7-E,Airport Operations,3/7/2023 0:00:00,0
2023-02-06,1408363,2,2023,02:30,Night,NaN,"Lanett, AL",NaN,NaN,06,...,NaN,NaN,NaN,NaN,REDACTED,REDACTED,FAA Form 5200-7-E,Air Transport Operations,3/16/2023 0:00:00,0
2023-02-14,1408923,2,2023,21:02,NaN,NaN,UNKNOWN/REMOTE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,REDACTED,REDACTED,FAA Form 5200-7-E,Airport Operations,3/22/2023 0:00:00,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 279947 entries, 1990-01-02 to 2023-05-17
Data columns (total 99 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   index_nr               279947 non-null  int64  
 1   incident_month         279947 non-null  int64  
 2   incident_year          279947 non-null  int64  
 3   time                   156370 non-null  object 
 4   time_of_day            162273 non-null  object 
 5   airport_id             279942 non-null  object 
 6   airport                279947 non-null  object 
 7   latitude               245560 non-null  float64
 8   longitude              245561 non-null  float64
 9   runway                 212020 non-null  object 
 10  state                  245563 non-null  object 
 11  faaregion              245563 non-null  object 
 12  location               32868 non-null   object 
 13  opid                   279943 non-null  object 
 14  operator            

In [11]:
for col in df.columns:
    print(f'Value Counts for {col}')
    print(df[col].value_counts())
    print(f'Num of nulls: {df[col].isnull().sum()}')
    print(f'Num of NON-nulls: {df[col].notnull().sum()}')
    print()

Value Counts for index_nr
1242675    1
793126     1
792678     1
792677     1
1091244    1
          ..
698690     1
700415     1
697820     1
698691     1
1427626    1
Name: index_nr, Length: 279947, dtype: int64
Num of nulls: 0
Num of NON-nulls: 279947

Value Counts for incident_month
8     39029
9     37479
7     35234
10    35116
5     26804
6     22884
4     20556
11    19292
3     13983
12    11078
1      9430
2      9062
Name: incident_month, dtype: int64
Num of nulls: 0
Num of NON-nulls: 279947

Value Counts for incident_year
2019    17342
2022    17203
2018    16205
2021    15643
2017    14777
2015    13773
2014    13692
2016    13327
2020    11624
2013    11410
2012    10932
2011    10111
2010     9897
2009     9504
2007     7740
2008     7627
2006     7273
2005     7243
2004     6566
2002     6217
2000     6025
2003     5989
2001     5822
1999     5120
1998     3809
1997     3559
2023     3042
1996     3031
1995     2825
1994     2708
1992     2651
1993     2624
1991     251

In [12]:
# decide which columns to keep
keep_cols = ['time_of_day', 'airport_id', 'airport', 'runway', 'state', 'opid', 'operator'
             , 'aircraft', 'ac_class', 'ac_mass', 'type_eng', 'num_engs', 'phase_of_flight'
             , 'precipitation', 'damage_level', 'species_id', 'species', 'size']
df = df[keep_cols]

In [13]:
# changing nulls to 'Unknown' for time of day (Dusk, Dawn, ..., Unknown)
df.time_of_day = df.time_of_day.fillna('Unknown')

In [14]:
df.time_of_day.value_counts()

Unknown    117674
Day         99664
Night       49431
Dusk         7287
Dawn         5891
Name: time_of_day, dtype: int64

In [15]:
df[df.airport_id.isnull()].shape

(5, 18)

In [16]:
df.shape

(279947, 18)

In [17]:
# there are only 5 nulls for airport_id, so I'm just going to remove them. It's easier than fixing them one at a time
df = df[df.airport_id.notnull()]
df.shape

(279942, 18)

In [18]:
df[df.airport_id == 'ZZZZ'].sample(3)

,time_of_day,airport_id,airport,runway,state,opid,operator,aircraft,ac_class,ac_mass,type_eng,num_engs,phase_of_flight,precipitation,damage_level,species_id,species,size
date,,,,,,,,,,,,,,,,,,
2011-02-20,Unknown,ZZZZ,UNKNOWN,NaN,NaN,EGF,AMERICAN EAGLE AIRLINES,EMB-145,A,3.0,D,2.0,NaN,None,N,UNKBS,Unknown bird - small,Small
2007-03-21,Night,ZZZZ,UNKNOWN,NaN,NaN,UPS,UPS AIRLINES,A-300,A,4.0,D,2.0,NaN,None,N,Z5101,Brown thrasher,Small
2007-07-21,Unknown,ZZZZ,UNKNOWN,NaN,NaN,DHL,ASTAR AIR CARGO,DC-9-40,A,4.0,D,2.0,NaN,NaN,N,UNKBS,Unknown bird - small,Small


In [19]:
# there are enough airport_id's with 'ZZZZ', i.e. the location of the birdstrike is unknown, that I want to save them
df.runway = np.where(df.runway.isnull(), '99', df.runway)

In [20]:
df.state = np.where(df.state.isnull(), 'ZZ', df.state)

In [21]:
df.phase_of_flight = np.where(df.phase_of_flight.isnull(), 'Unknown', df.phase_of_flight)

In [23]:
df.precipitation = np.where(df.precipitation.isnull(), 'Unknown', df.precipitation)

In [24]:
df.head()

,time_of_day,airport_id,airport,runway,state,opid,operator,aircraft,ac_class,ac_mass,type_eng,num_engs,phase_of_flight,precipitation,damage_level,species_id,species,size
date,,,,,,,,,,,,,,,,,,
1990-01-02,Unknown,KJFK,JOHN F KENNEDY INTL,31L,NY,FDX,FEDEX EXPRESS,DC-10,A,4.0,D,3.0,Departure,Unknown,N,UNKB,Unknown bird,NaN
1990-01-02,Unknown,PHLI,LIHUE ARPT,99,HI,UNK,UNKNOWN,UNKNOWN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,R1101,Barn owl,Small
1990-01-04,Unknown,PHLI,LIHUE ARPT,99,HI,HAL,HAWAIIAN AIR,DC-9,A,4.0,D,2.0,Landing Roll,Unknown,NaN,ZY1,Munias,Small
1990-01-05,Unknown,PHLI,LIHUE ARPT,99,HI,UNK,UNKNOWN,UNKNOWN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,N5122,Pacific golden-plover,Small
1990-01-05,Unknown,PHLI,LIHUE ARPT,99,HI,UNK,UNKNOWN,UNKNOWN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,ZY1,Munias,Small


In [ ]:
##  START BACK UP HERE:  Deal with remaining nulls, then do some univariate plotting of counts over time

In [25]:
for col in df.columns:
    print(f'Value Counts for {col}')
    print(df[col].value_counts())
    print(f'Num of nulls: {df[col].isnull().sum()}')
    print(f'Num of NON-nulls: {df[col].notnull().sum()}')
    print()

Value Counts for time_of_day
Unknown    117672
Day         99664
Night       49430
Dusk         7286
Dawn         5890
Name: time_of_day, dtype: int64
Num of nulls: 0
Num of NON-nulls: 279942

Value Counts for airport_id
ZZZZ    34302
KDEN     9260
KDFW     7527
KORD     6065
KJFK     5882
        ...  
T40         1
LIRP        1
KTLR        1
OI65        1
PALP        1
Name: airport_id, Length: 2591, dtype: int64
Num of nulls: 0
Num of NON-nulls: 279942

Value Counts for airport
UNKNOWN                        34302
DENVER INTL AIRPORT             9260
DALLAS/FORT WORTH INTL ARPT     7527
CHICAGO O'HARE INTL ARPT        6065
JOHN F KENNEDY INTL             5882
                               ...  
HOUSTON GULF ARPT                  1
GALILEO GALILEI ARPT               1
MCGEHEE CATFISH REST               1
MEFFORD FIELD ARPT                 1
ALPINE AIRSTRIP                    1
Name: airport, Length: 2591, dtype: int64
Num of nulls: 0
Num of NON-nulls: 279942

Value Counts for runwa